In [1]:

from IPython.display import display

In [2]:
import ipywidgets as widgets
import tempfile
import os, shutil

In [3]:
import subprocess

In [4]:
tmpdir = tempfile.mkdtemp(suffix=None, prefix=None, dir=None)

In [5]:
ROOT=os.getcwd()

In [6]:
#default style
style = {'description_width': 'initial'}

layout=widgets.Layout(width='80%')

In [7]:
# input files widget
sequence_file_upload_widget = widgets.FileUpload(
                        accept='fas, fasta, fa',
                        multiple=True,
                        disabled=False,
                        description="Select sequence files (.fas)",
                        style=style,
                        layout=layout
)

reference_strain_widget = widgets.Dropdown(
                            options=[],
                            description="Select the reference strain.",
                            disabled=True,
                            style=style,
                            layout=layout,
)

def on_sequence_file_upload_change(change):
    # Write uploded files to tmpdir
    


    for strain in change['new'].values():
        with open(os.path.join(tmpdir, strain['metadata']['name']), 'wb') as fp:
            fp.write(strain['content'])
            
    # Set options on reference_strain_widget
    reference_strain_widget.options=list(change['new'].keys())
    reference_strain_widget.disabled = False

sequence_file_upload_widget.observe(on_sequence_file_upload_change, names='value')

In [8]:
# parameter widgets
min_nmer_widget = widgets.IntSlider(value=55,
                                            min=0,
                                            max=100,
                                            description="Min nmer occurrence",
                                            style=style,  
                                            layout=layout,
                                   )


nmer_length_widget = widgets.IntSlider(value=21,
                                            min=0,
                                            max=100,
                                            description="Nmer length",
                                            style=style,
                                            layout=layout,
                                      )

In [9]:
query_rayt_upload_widget = widgets.FileUpload(
    accept="faa",
    multiple=False,
    description="Select query rayt sequence file (.faa)",
    style=style,
    layout=layout,
)

def on_query_rayt_upload_change(change):
    # Write uploded files to tmpdir

    for strain in change['new'].values():
        with open(os.path.join(tmpdir, 'query_rayt.faa'), 'wb') as fp:
            fp.write(strain['content'])
            

query_rayt_upload_widget.observe(on_query_rayt_upload_change, names='value')

In [10]:
tree_file_widget = widgets.Text(value="strain.nwk",
                                         placeholder="Name of tree file",
                                         description="Name (path) of the tree file. Will be generated if not present.",
                                         disabled=False,
                                        style=style,
                                        layout=layout
                               )

In [11]:
e_widget = widgets.FloatText(value=1e-90,
                             description="e value cutoff",
                             disabled=False,
                             style=style,
                       )

In [12]:
def check_all_inputs():
    return all([True])

In [13]:
output_widget = widgets.Output()

In [14]:
run_button = widgets.Button(
    description='Run RepinPop analysis',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run the Repin Population analysis',
    icon='run'
)

In [15]:
def on_button_clicked(b):
    with output_widget:
        print("RepinPop starts")
        run_repin_pop()

In [16]:
run_button.on_click(on_button_clicked)

In [17]:
def run_repin_pop():
    oldwd = os.getcwd()
    #os.chdir(tempfile.gettempdir())
    os.chdir('/')
    command = ['java',
              '-jar',
              os.path.abspath(
                  os.path.join(ROOT,
                               '..',
                               'REPIN_ecology/REPIN_ecology/build/libs/REPIN_ecology.jar')),
              tmpdir,
               reference_strain_widget.value,
               '{0:d}'.format(min_nmer_widget.value),
               '{0:d}'.format(nmer_length_widget.value),
               os.path.join(tmpdir, 'query_rayt.faa'),
               os.path.join(tmpdir, tree_file_widget.value),
               '{0:1.2e}'.format(e_widget.value)
              ]
    
    
    #print(command)
    
    with subprocess.Popen(command,
                          stdout=subprocess.PIPE,
                          stderr=subprocess.STDOUT) as proc:
        
        
        print(proc.stdout.read())
            

    
    os.chdir(oldwd)
    

In [18]:
gui = widgets.VBox(
    [
    widgets.VBox(
        [
        sequence_file_upload_widget,
        reference_strain_widget,
        ]
    ),
        query_rayt_upload_widget,
        widgets.VBox(
            [
                min_nmer_widget,
                nmer_length_widget,
            ]
        ),
        widgets.VBox(
            [
                tree_file_widget,
                e_widget,
            ]
        ),
        run_button,
        output_widget
    ],
    #layout=widgets.Layout(width='100%')
)

In [19]:
_ = display(gui)